In [30]:
import json

In [31]:
import mysql.connector
from mysql.connector import Error

In [32]:
try:
    connection = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='password'
    )
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS Twitterbase")
except Error as e:
    print("Error while connecting to MySQL", e)


Connected to MySQL Server version  8.0.36


In [33]:
cursor.execute("USE Twitterbase")

In [34]:
import datetime

In [35]:
# Creating the USER_INFO table
cursor.execute("""
CREATE TABLE IF NOT EXISTS USER_INFO (
    ID BIGINT PRIMARY KEY,
    NAME VARCHAR(255),
    SCREEN_NAME VARCHAR(255),
    VERIFIED BOOLEAN,
    FOLLOWERS_COUNT INT,
    PROFILE_IMAGE_URL TEXT
);
""")

In [36]:
connection.commit()

In [37]:
def insert_user(data, cursor):
    user_data = data['user']
    user_values = (
        user_data['id'],
        user_data['name'],
        user_data['screen_name'],
        user_data['verified'],
        user_data['followers_count'],
        user_data['profile_image_url']
    )
    insert_query = """
    INSERT INTO USER_INFO (ID, NAME, SCREEN_NAME, VERIFIED, FOLLOWERS_COUNT, PROFILE_IMAGE_URL)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    NAME = VALUES(NAME),
    SCREEN_NAME = VALUES(SCREEN_NAME),
    VERIFIED = VALUES(VERIFIED),
    FOLLOWERS_COUNT = VALUES(FOLLOWERS_COUNT),
    PROFILE_IMAGE_URL = VALUES(PROFILE_IMAGE_URL)
    """
    cursor.execute(insert_query, user_values)

In [38]:
with open("corona-out-3", "r") as file:
    for line_number, line in enumerate(file, 1):
        try:
            if not line.strip():
                continue
            data = json.loads(line)
            insert_user(data, cursor)
        except json.JSONDecodeError:
            print(f"Error processing line {line_number}: Invalid JSON")
        except Exception as e:
            print(f"Error processing line {line_number}: {e}")

In [39]:
connection.commit()

In [40]:
cursor.execute("SELECT * FROM User_info LIMIT 5;")
print("First 5 entries from the Users table:")
for row in cursor.fetchall():
    print(row)

First 5 entries from the Users table:
(767, 'Xeni Jardin 😷🏠', 'xeni', 1, 152608, 'http://pbs.twimg.com/profile_images/1247496153878282240/g358Pjaq_normal.jpg')
(5039, 'Jrome', 'jrome', 0, 642, 'http://pbs.twimg.com/profile_images/1035729023110971392/tdGGy2z5_normal.jpg')
(5242, 'Ahmed Al Omran', 'ahmed', 1, 149844, 'http://pbs.twimg.com/profile_images/1175469297675714562/QUWgZ3Gk_normal.jpg')
(11364, 'Kirti', 'Kits', 0, 922, 'http://pbs.twimg.com/profile_images/1236533691511214080/-RiWXOon_normal.jpg')
(12094, 'Nicholai Reinseth', 'Nicholai', 0, 277, 'http://pbs.twimg.com/profile_images/792446254236233728/86sl8Ero_normal.jpg')


In [41]:
def construct_query(searchterm, lim, metric):
    base_query = ""
    where_clause = ""
    if lim:
        if lim <=20:
            limit_clause = f" LIMIT {lim}"
    else:
        lim = 20
        limit_clause = f" LIMIT {lim}"
    
    # Determine the base query based on the metric
    if metric == "users":
        base_query = "SELECT * FROM USERS"
        # Always sort users by followers count in descending order
        order_clause = " ORDER BY FOLLOWERS_COUNT DESC"
        # If a search term is provided, add a WHERE clause to filter by NAME
        if searchterm:
            where_clause = f" WHERE NAME LIKE '%{searchterm}%'"
    
    # Combine the parts of the query
    query = f"{base_query}{where_clause}{order_clause}{limit_clause};"
    
    return query


In [42]:
# Example 1: Search for users by name, limit to 10 results
query1 = construct_query("John", 10, "users")
print(query1)

# Example 2: Get top 5 users without search term
query2 = construct_query("", 5, "users")
print(query2)

# Example 3: Search for users by name without limit
query3 = construct_query("Doe", None, "users")
print(query3)


SELECT * FROM USERS WHERE NAME LIKE '%John%' ORDER BY FOLLOWERS_COUNT DESC LIMIT 10;
SELECT * FROM USERS ORDER BY FOLLOWERS_COUNT DESC LIMIT 5;
SELECT * FROM USERS WHERE NAME LIKE '%Doe%' ORDER BY FOLLOWERS_COUNT DESC LIMIT 20;


In [43]:
cursor.execute(query1)
print("query1")
for row in cursor.fetchall():
    print(row)

query1
(259395895, 'John Harwood', 1, 363458, 'http://pbs.twimg.com/profile_images/868178337859911680/GJ8zcy-z_normal.jpg')
(558686364, 'John Heche', 0, 213085, 'http://pbs.twimg.com/profile_images/597395337133871104/HMhVGGX3_normal.jpg')
(457320286, 'John A. A. Logan', 0, 77906, 'http://pbs.twimg.com/profile_images/1745878945/thomasford800by533_edited-1_normal.jpg')
(711627609344516096, 'John Trumpfan', 0, 43994, 'http://pbs.twimg.com/profile_images/853683089280827392/E_dLFBgK_normal.jpg')
(1171793868, 'john jackson', 0, 42866, 'http://pbs.twimg.com/profile_images/838235597080674304/37K1h01M_normal.jpg')
(847845464489824256, 'Rodney Johnston MAGA text TRUMP to 88022', 0, 39394, 'http://pbs.twimg.com/profile_images/1171090294944817152/TxLWkZNp_normal.jpg')
(4910826454, 'Elnathan John', 1, 38918, 'http://pbs.twimg.com/profile_images/1249825749244182537/-5T7M4Pe_normal.jpg')
(239432753, 'John Weant', 0, 20596, 'http://pbs.twimg.com/profile_images/1221586610774474752/Xh2j5NWl_normal.jpg')

In [44]:
cursor.execute(query2)
print("query2")
for row in cursor.fetchall():
    print(row)

query2
(69183155, 'detikcom', 1, 15928061, 'http://pbs.twimg.com/profile_images/1253960978305183746/Yz0vPk8w_normal.jpg')
(62513246, 'J.K. Rowling', 1, 14608046, 'http://pbs.twimg.com/profile_images/1240994661323411456/9CeA8lPP_normal.jpg')
(42606652, 'AajTak', 1, 9706667, 'http://pbs.twimg.com/profile_images/1179039747273940992/ZGQ_ibsP_normal.jpg')
(39240673, 'ABP News', 1, 9563509, 'http://pbs.twimg.com/profile_images/1093087956141715456/uR5Dz8p2_normal.jpg')
(240649814, 'TIMES NOW', 1, 9499855, 'http://pbs.twimg.com/profile_images/548508720888442881/T4tYT9xR_normal.jpeg')


In [45]:
cursor.execute(query3)
print("query3")
for row in cursor.fetchall():
    print(row)

query3
(16188864, 'M. Balter teaches/does journalism, remotely.', 0, 6014, 'http://pbs.twimg.com/profile_images/769528655760547841/AOg5UXnh_normal.jpg')
(271883332, 'Karl Doemens', 0, 4147, 'http://pbs.twimg.com/profile_images/1214309595553943553/FIoo5VRm_normal.jpg')
(1060837449147076609, '🇬🇧🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007fRoger Beddoe🇬🇧🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f', 0, 4003, 'http://pbs.twimg.com/profile_images/1247455175658397696/yRqZNttX_normal.jpg')
(915678870786916353, 'E. Valencia #YoMeQuedoEnCasa🙂🤖🤖', 0, 2568, 'http://pbs.twimg.com/profile_images/1237233271613706240/wF_ta_ye_normal.jpg')
(898651479887818753, 'Jean-Pierre DOEUFF', 0, 2423, 'http://pbs.twimg.com/profile_images/1152220398773243904/XuT5n4Sw_normal.jpg')
(747872377, 'Jessica Doering', 0, 2385, 'http://pbs.twimg.com/profile_images/1143632065860186112/N2ZgmrSh_normal.jpg')
(496552854, 'IkDoeWatIkDoe', 0, 1235, 'http://pbs.twimg.com/profile_images/1021498037

# Everything below is previous version stuff:

In [46]:
#dropped old tables
create_users_table_query = """
    CREATE TABLE IF NOT EXISTS USERS(
        ID BIGINT PRIMARY KEY,
        NAME VARCHAR(255),
        VERIFIED BOOLEAN,
        FOLLOWERS_COUNT INT,
        PROFILE_IMAGE_URL TEXT
    );
    """
    
# Execute the query to create the Users table
cursor.execute(create_users_table_query)

In [47]:
connection.commit()

In [48]:
def insert_or_update_user(user_data, cursor):
    user_values = (
        user_data['id'],
        user_data['name'],
        user_data['verified'],
        user_data['followers_count'],
        user_data['profile_image_url']
    )
    insert_query = """
    INSERT INTO USERS (ID, NAME, VERIFIED, FOLLOWERS_COUNT, PROFILE_IMAGE_URL)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    NAME = VALUES(NAME),
    VERIFIED = VALUES(VERIFIED),
    FOLLOWERS_COUNT = VALUES(FOLLOWERS_COUNT),
    PROFILE_IMAGE_URL = VALUES(PROFILE_IMAGE_URL);
    """
    cursor.execute(insert_query, user_values)


In [49]:
with open("corona-out-3", "r") as file:
    for line_number, line in enumerate(file, 1):
        try:
            if not line.strip():
                continue
            data = json.loads(line)
            # Process user data here for both tweets and retweets
            if 'user' in data:
                insert_or_update_user(data['user'], cursor)

        except json.JSONDecodeError:
            print(f"Error processing line {line_number}: Invalid JSON")
        except Exception as e:
            print(f"Error processing line {line_number}: {e}")

In [50]:
connection.commit()

In [51]:
cursor.execute("SELECT * FROM Users LIMIT 5;")
print("First 5 entries from the Users table:")
for row in cursor.fetchall():
    print(row)

First 5 entries from the Users table:
(767, 'Xeni Jardin 😷🏠', 1, 152608, 'http://pbs.twimg.com/profile_images/1247496153878282240/g358Pjaq_normal.jpg')
(5039, 'Jrome', 0, 642, 'http://pbs.twimg.com/profile_images/1035729023110971392/tdGGy2z5_normal.jpg')
(5242, 'Ahmed Al Omran', 1, 149844, 'http://pbs.twimg.com/profile_images/1175469297675714562/QUWgZ3Gk_normal.jpg')
(11364, 'Kirti', 0, 922, 'http://pbs.twimg.com/profile_images/1236533691511214080/-RiWXOon_normal.jpg')
(12094, 'Nicholai Reinseth', 0, 277, 'http://pbs.twimg.com/profile_images/792446254236233728/86sl8Ero_normal.jpg')


In [52]:
connection.close()